# Family Budget Calculator

Goal: Measure monthly income needed by a family to maintain a reasonable standard of living.
Family size options: 1-2 adults, 0-4 children (view assumptions about families in [source](https://www.epi.org/publication/family-budget-calculator-documentation/))

Budget Components:
- Housing
- Food
- Transportation
- Child care
- Health care
- Taxes
- Other necessities



In [1]:
import numpy as np
import pandas as pd
from datetime import date

today = date.today()

In [2]:
!java --version

java 15.0.2 2021-01-19
Java(TM) SE Runtime Environment (build 15.0.2+7-27)
Java HotSpot(TM) 64-Bit Server VM (build 15.0.2+7-27, mixed mode, sharing)


## Food
Data sources:
- National average food costs [Official USDA Food Plans: Cost of Food Reports](https://www.fns.usda.gov/cnpp/usda-food-plans-cost-food-reports-monthly-reports)  
Note: "USDA suggests making the following adjustments to account for differences in returns to scale:
 - One-person family: add 20 percent
 - Two-person family: add 10 percent
 - Three-person family: add 5 percent
 - Five-person family: subtract 5 percent
 - Six-person family: subtract 5 percent"
  
- County-level multipliers [Feeding America, Map the Meal Gap](https://map.feedingamerica.org/)

### USDA Food Plans: Cost of Food

In [3]:
households = ['Individuals', 'Families']
groups = ['Child', 'Male', 'Female', 'Family of 2', 'Family of 4']

types = ['Thrifty', 'Low-cost', 'Moderate', 'Liberal']

In [4]:
import requests
import tabula
import pandas as pd
from dateutil.relativedelta import relativedelta

lastDate = today
increment = relativedelta(months = 1)

print('Scanning for latest data...')

for i in range(10):
    print('>', str.ljust(lastDate.strftime('%B %Y'), 16), end = '')
    
    file_end = f'{lastDate.strftime("%b")}{today.year}'
    url = f'https://fns-prod.azureedge.net/sites/default/files/media/file/CostofFood{file_end}.pdf'
    req = requests.get(url, stream=True)
    
    print('status', req.status_code)
    
    if req.status_code == 200:
        break
    else:
        lastDate -= increment

print(f'Read "{url}"...', end='')
tables = tabula.read_pdf(url,
                        pages = 1,
                        multiple_tables = False,
                        guess = True,
                        stream = True,
                        pandas_options = {'header':1},
                       )
print('done')
df = tables[0]

Scanning for latest data...
> March 2021      status 404
> February 2021   status 404
> January 2021    status 200
Read "https://fns-prod.azureedge.net/sites/default/files/media/file/CostofFoodJan2021.pdf"...done


In [5]:
import re

#for finding cost values from df row
pattern = re.compile(r'[^(.$\d)]')

#for matching table groups
group_patterns = [re.compile(group.replace(' ', '.+')) for group in groups]

#
n_types = len(types)

#
na = df[df.isna().any(axis = 1)].iloc[:,0]
na = na.to_numpy()

#for iteration
foodCosts = dict()
inKey = ''
newKey = ''

#assuming weekly and monthly data available
foodCosts['week'] = dict()
foodCosts['month'] = dict()

#loop
keys = df.iloc[:,0].dropna()
for i, row in zip(keys.index, keys.values):
    
    flag = row in na
    ignore = any([household in row for household in households])
    
    if not ignore:
        
        if flag: # get demographic
            inKey += ' ' + row.encode('ascii','ignore').decode() #strip non-ascii chars
        
        else:
            if inKey != '': # create df from demographic
                
                for j in [3, 4, 0, 1, 2]:
                    if group_patterns[j].search(inKey.strip(' :4')):
                        newKey = groups[j]
                        break
                
                foodCosts['week'][newKey] = pd.DataFrame(columns = types)
                foodCosts['month'][newKey] = pd.DataFrame(columns = types)
                inKey = ''
            
            string = df.iloc[i,1:].to_string(header=False, index=False).strip(' $')
            values = re.sub(pattern, '', string).split('$')
            
            assert (len(values) == 2*n_types)
            
            newWeekly = pd.DataFrame(index = [row], columns = types, data = [values[:n_types]], dtype = float)
            newMonthly = pd.DataFrame(index = [row], columns = types, data = [values[n_types:]], dtype = float)
            
            foodCosts['week'][newKey] = pd.concat([foodCosts['week'][newKey], newWeekly])
            foodCosts['month'][newKey] = pd.concat([foodCosts['month'][newKey], newMonthly])

def getRowFromAge(df:pd.DataFrame, age:int) -> pd.Series:
    for idx in df.index:
        ages = re.findall(r'\d+', idx)
        ages = list(map(int, ages))
        
        if ages[0] <= age <= ages[-1]: #age in range of index
            return df.loc[idx]

USDA suggested scaling relative to household size

In [6]:
adjustment = {
    1 : 1.2,
    2 : 1.1,
    3 : 1.05,
    4 : 1,
    5 : 0.95,
    6 : 0.95,
}

# note: max age 71
def getFoodCost(people:list, timeframe:str = 'week', plan = 'Moderate') -> pd.Series:

    assert 1 <= len(people) <= 6
    assert timeframe == 'week' or timeframe == 'month' or timeframe == 'year'
    
    df = foodCosts['week' if timeframe == 'week' else 'month']
    total = 0
    multiplier = adjustment[len(people)]
    
    for group, age in people:
        if group == 'm' or group == 'male':
            assert age >= 12
            total += getRowFromAge(df['Male'], age)

        elif group == 'f' or group == 'female':
            assert age >= 12
            total += getRowFromAge(df['Female'], age)
            
        elif group == 'c' or group == 'child':
            assert age < 12
            total += getRowFromAge(df['Child'], age)
    
    if timeframe == 'year':
        total *= 12
    
    total.name = f'Food Costs ({timeframe}ly)'
    
    return total * multiplier

Test food calculator with sample 3-person family

In [7]:
getFoodCost([['m',30],['f',30],['c', 5]], timeframe = 'year')

Thrifty      6051.78
Low-cost     7731.36
Moderate     9589.86
Liberal     11895.66
Name: Food Costs (yearly), dtype: float64